##Data loading

Open Sourcing Mental Illness (OSMI) data: https://www.kaggle.com/datasets/osmi/mental-health-in-tech-survey

This dataset contains the following data:

* Timestamp
* Age
* Gender
* Country
* state: If you live in the United States, which state or territory do you live in?
* self_employed: Are you self-employed?
* family_history: Do you have a family history of mental illness?
* treatment: Have you sought treatment for a mental health condition?
* work_interfere: If you have a mental health condition, do you feel that it interferes with your work
* no_employees: How many employees does your company or organization have?
* remote_work: Do you work remotely (outside of an office) at least 50% of the time?
* tech_company: Is your employer primarily a tech company/organization?
* benefits: Does your employer provide mental health benefits?
* care_options: Do you know the options for mental health care your employer provides?
* wellness_program: Has your employer ever discussed mental health as part of an employee wellness program?
* seek_help: Does your employer provide resources to learn more about mental health issues and how to seek help?
* anonymity: Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources?
* leave: How easy is it for you to take medical leave for a mental health condition?
* mentalhealthconsequence: Do you think that discussing a mental health issue with your employer would have negative consequences?
* physhealthconsequence: Do you think that discussing a physical health issue with your employer would have negative consequences?
* coworkers: Would you be willing to discuss a mental health issue with your coworkers?
* supervisor: Would you be willing to discuss a mental health issue with your direct supervisor(s)
* mentalhealthinterview: Would you bring up a mental health issue with a potential employer in an interview?* physhealthinterview: Would you bring up a physical health issue with a potential employer in an interview?
* mentalvsphysical: Do you feel that your employer takes mental health as seriously as physical health?
* obs_consequence: Have you heard of or observed negative consequences for coworkers with mental health conditions in your workplace?
* comments: Any additional notes or comments



In [1]:
 # load libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter
from statistics import mean, stdev

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold as skf

In [2]:
#mount Google Drive to access the folders
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
#import data from our data folder
import pandas as pd
dataStreamPath = '/content/drive/MyDrive/bipolar_shap/data/'

In [4]:
# load data
df = pd.read_csv(dataStreamPath + "survey.csv")

In [5]:
#folders where we will save the shap graphs and values, and linguistic summaries
#please create these folders in your Google Drive
graphsStreamPath='/content/drive/MyDrive/bipolar_shap/graphs/'
shapvaluesStreamPath='/content/drive/MyDrive/bipolar_shap/shapvalues/'
ResultsDir = '/content/drive/MyDrive/bipolar_shap/LS-XAI/'

In [6]:
df

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,46,f,United States,NC,No,No,No,NaN,100-500,...,Don't know,Yes,No,No,No,No,No,No,No,NaN


In [7]:
#count instances for each attribute
df.count()

Timestamp                    1259
Age                          1259
Gender                       1259
Country                      1259
state                         744
self_employed                1241
family_history               1259
treatment                    1259
work_interfere                995
no_employees                 1259
remote_work                  1259
tech_company                 1259
benefits                     1259
care_options                 1259
wellness_program             1259
seek_help                    1259
anonymity                    1259
leave                        1259
mental_health_consequence    1259
phys_health_consequence      1259
coworkers                    1259
supervisor                   1259
mental_health_interview      1259
phys_health_interview        1259
mental_vs_physical           1259
obs_consequence              1259
comments                      164
dtype: int64

## Data preprocessing

In [8]:
# remove unnecessary features
df = df.drop(['Timestamp', 'Country', 'state', 'comments'], axis=1)

In [9]:
#count unique values for each attribute
for column in df:
    print(df[column].value_counts())

 29             85
 32             82
 26             75
 27             71
 33             70
 28             68
 31             67
 34             65
 30             63
 25             61
 35             55
 23             51
 24             46
 37             43
 38             39
 36             37
 40             33
 39             33
 43             28
 22             21
 41             21
 42             20
 21             16
 45             12
 46             12
 44             11
 19              9
 18              7
 48              6
 50              6
 20              6
 51              5
 49              4
 56              4
 57              3
 54              3
 55              3
 47              2
 60              2
 99999999999     1
 5               1
-1               1
 11              1
 8               1
 61              1
 53              1
-29              1
-1726            1
 65              1
 62              1
 58              1
 329             1
 72         

In [10]:
df['Age'].unique()

array([         37,          44,          32,          31,          33,
                35,          39,          42,          23,          29,
                36,          27,          46,          41,          34,
                30,          40,          38,          50,          24,
                18,          28,          26,          22,          19,
                25,          45,          21,         -29,          43,
                56,          60,          54,         329,          55,
       99999999999,          48,          20,          57,          58,
                47,          62,          51,          65,          49,
             -1726,           5,          53,          61,           8,
                11,          -1,          72])

In [11]:
# remove strange age values
df = df[df['Age'] != -29]
df = df[df['Age'] != 329]
df = df[df['Age'] != 99999999999]
df = df[df['Age'] != -1726]
df = df[df['Age'] != 5]
df = df[df['Age'] != 8]
df = df[df['Age'] != -1]

In [12]:
df['Gender'].unique()

array(['Female', 'M', 'Male', 'male', 'female', 'm', 'Male-ish', 'maile',
       'Trans-female', 'Cis Female', 'F', 'something kinda male?',
       'Cis Male', 'Woman', 'f', 'Mal', 'Male (CIS)', 'queer/she/they',
       'non-binary', 'Femake', 'woman', 'Make', 'Nah', 'Enby', 'fluid',
       'Genderqueer', 'Female ', 'Androgyne', 'Agender',
       'cis-female/femme', 'Guy (-ish) ^_^', 'male leaning androgynous',
       'Male ', 'Man', 'Trans woman', 'msle', 'Neuter', 'Female (trans)',
       'queer', 'Female (cis)', 'Mail', 'cis male', 'Malr', 'femail',
       'Cis Man', 'ostensibly male, unsure what that really means'],
      dtype=object)

In [13]:
# standardize gender
error = {'Female':'F',
         'Male':'M',
         'male':'M',
         'female':'F',
         'm':'M',
         'Male-ish':'M',
         'maile':'M',
         'Trans-female':'T',
         'Cis Female':'F',
         'something kinda male?':'M',
         'Cis Male':'M',
         'Woman':'F',
         'f':'F',
         'Mal':'M',
         'Male (CIS)':'M',
         'queer/she/they':'F',
         'non-binary':'T',
         'Enby':'T',
         'Femake':'F',
         'woman':'F',
         'Make':'M',
         'fluid':'T',
         'Malr':'M',
         'cis male':'M',
         'Female (cis)':'F',
         'Guy (-ish) ^_^':'M',
         'queer':'T',
         'Female (trans)':'T',
         'male leaning androgynous':'T',
         'Neuter':'T',
         'cis-female/femme':'F',
         'msle':'M',
         'Agender':'T',
         'Genderqueer':'T',
         'Female':'F',
         'Androgyne':'T',
         'Nah':'T', 
         'All':'T',
         'Female ':'F',
         'Male ':'M', 
         'Man':'M', 
         'Trans woman':'T', 
         'Mail':'M',
         'A little about you':'T',
         'femail': 'F',
         'Cis Man': 'M',
         'ostensibly male, unsure what that really means': 'M',
         'p': 'T'}

df['Gender'] = df['Gender'].map(error).fillna(df['Gender'])

In [14]:
df['Gender'].unique()

array(['F', 'M', 'T'], dtype=object)

In [15]:
df = df.dropna()

In [16]:
df.count()

Age                          972
Gender                       972
self_employed                972
family_history               972
treatment                    972
work_interfere               972
no_employees                 972
remote_work                  972
tech_company                 972
benefits                     972
care_options                 972
wellness_program             972
seek_help                    972
anonymity                    972
leave                        972
mental_health_consequence    972
phys_health_consequence      972
coworkers                    972
supervisor                   972
mental_health_interview      972
phys_health_interview        972
mental_vs_physical           972
obs_consequence              972
dtype: int64

In [17]:
for column in df:
    print(df[column].value_counts())

32    66
29    60
26    60
28    54
34    53
30    52
27    51
33    51
31    47
25    46
35    44
23    39
37    33
24    32
36    32
38    31
40    29
39    25
43    20
41    18
42    16
22    15
21    14
45    11
46    11
44     8
19     6
18     5
48     5
20     5
56     4
51     4
50     4
49     4
54     3
55     3
57     3
61     1
72     1
11     1
62     1
53     1
47     1
58     1
60     1
Name: Age, dtype: int64
M    751
F    207
T     14
Name: Gender, dtype: int64
No     850
Yes    122
Name: self_employed, dtype: int64
No     533
Yes    439
Name: family_history, dtype: int64
Yes    619
No     353
Name: treatment, dtype: int64
Sometimes    457
Never        207
Rarely       170
Often        138
Name: work_interfere, dtype: int64
26-100            223
More than 1000    220
6-25              212
1-5               137
100-500           137
500-1000           43
Name: no_employees, dtype: int64
No     677
Yes    295
Name: remote_work, dtype: int64
Yes    795
No     177
Name: te

In [18]:
# class labels
y = df['treatment'].values

In [19]:
# categorical to numeric
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [20]:
# treatment 0: No 1:Yes 
Counter(y)

Counter({0: 353, 1: 619})

In [21]:
X = df.drop(['treatment'], axis=1).values

In [22]:
# categorical to numeric
enc = preprocessing.OrdinalEncoder()
enc.fit(X)
X = enc.transform(X)
X

array([[29.,  1.,  1., ...,  2.,  2.,  1.],
       [12.,  1.,  0., ...,  1.,  1.,  0.],
       [14.,  1.,  1., ...,  0.,  2.,  0.],
       ...,
       [15.,  1.,  0., ...,  1.,  2.,  0.],
       [17.,  1.,  0., ...,  1.,  1.,  0.],
       [ 8.,  1.,  0., ...,  1.,  0.,  0.]])

In [23]:
# scaling
scaler = preprocessing.MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
X

array([[0.65909091, 0.5       , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.27272727, 0.5       , 0.        , ..., 0.5       , 0.5       ,
        0.        ],
       [0.31818182, 0.5       , 1.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.34090909, 0.5       , 0.        , ..., 0.5       , 1.        ,
        0.        ],
       [0.38636364, 0.5       , 0.        , ..., 0.5       , 0.5       ,
        0.        ],
       [0.18181818, 0.5       , 0.        , ..., 0.5       , 0.        ,
        0.        ]])

In [24]:
# cross-validation
skf = StratifiedKFold(n_splits=10)

In [25]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [26]:
feature_names=df.drop(['treatment'], axis=1).columns

In [27]:
#feature_names

In [28]:
classes_names=list(set(le.inverse_transform(y_test)))

In [29]:
# final classes
classes_names

['Yes', 'No']

In [30]:
X_test_df=pd.DataFrame(X_test,columns=feature_names)

In [31]:
#I keep the not normalized data for Linguistic explanations
X_test_nn=scaler.inverse_transform(X_test_df)
X_test_df_nn=pd.DataFrame(X_test_nn,columns=feature_names)
X_test_df_nn['class']=y_test

In [32]:
X_test_df['class']=y_test

We will use Shap graphs to explain black-box algorithms in terms of features contributions to the results (https://shap-lrjball.readthedocs.io/en/docs_update/index.html)

In [33]:
! pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 13.8 MB/s 


In [34]:
import shap

#Linguistic summaries - functions definitions



In [35]:
%pip install -U scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 993 kB 13.6 MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894089 sha256=02cb7f0795d70851e351a11d486bc398cee46b0452fc3a52c7afeb13232b0a6f
  Stored in directory: /root/.cache/pip/wheels/d5/74/fc/38588a3d2e3f34f74588e6daa3aa5b0a322bd6f9420a707131
Successfully built scikit-fuzzy


In [36]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from copy import deepcopy
import warnings
import seaborn as sns

#function definition
def membership_function(data, var_name, value, central=0, spread=0.1, plot=False, na_omit=False,expert = False,use_central_and_spread=False):
    d = deepcopy(data)
    
    if na_omit:
        d = d.loc[~d[var_name].isna()]
    else:
        d = d.fillna(0)
        
    d = d[var_name]
    
    max_for_universe = np.max(d)  
    min_for_universe = np.min(d)

    universe = np.arange(min_for_universe, max_for_universe, 0.001)
    reg_name = var_name 
    reg = ctrl.Consequent(universe, reg_name)

    ###################################################
    #print "central value:"
    if expert:
        #print ('central value', central)
        median_pos_t = np.mean(d>central)
        median_neg_t = np.mean(d<central)
        median_pos = central+median_pos_t
        median_neg = central-median_neg_t
    
    if use_central_and_spread:
          first_quartile = np.max([median_neg,min_for_universe])
          median_quartile = central
          #third_quartile = np.min([central+(spread),max_for_universe])
          third_quartile = np.min([median_pos,max_for_universe])
    else:        
          first_quartile = np.percentile(d, 25)
          median_quartile = np.percentile(d, 50)
          third_quartile = np.percentile(d, 75)
    
    if spread=="100":
         first_quartile = (max_for_universe-min_for_universe)/3
         median_quartile = (max_for_universe-min_for_universe)/2
         third_quartile = 2*(max_for_universe-min_for_universe)/3
 
       
   #quartiles based fuzzification
    #print('min_for_universe', min_for_universe)
    #print('first_quartile',first_quartile)
    #print('median_quartile',median_quartile)
    #print('third_quartile',third_quartile)
    #print('max_for_universe',max_for_universe)
    low = fuzz.trapmf(reg.universe, [min_for_universe, min_for_universe, first_quartile, median_quartile])
    medium = fuzz.trimf(reg.universe, [first_quartile, median_quartile, third_quartile])
    high = fuzz.trapmf(reg.universe, [median_quartile, third_quartile, max_for_universe, max_for_universe])
     
    if plot:     
        fig, (ax0) = plt.subplots(nrows=1, figsize=(5, 3))
        ax0.plot(universe, low, 'b', linewidth=2, label='low')
        ax0.plot(universe, medium, 'r', linewidth=2, label='medium')
        ax0.plot(universe, high, 'g', linewidth=2, label='high')
        ax0.set_title(str(var_name))
        ax0.legend()
        plt.tight_layout()
        fig.savefig(ResultsDir+"LinguisticVariable_"+str(var_name)+"_spread_"+str(spread)+".png")
        plt.close()
        #quit()

    return (fuzz.interp_membership(universe, low, value),
            fuzz.interp_membership(universe, medium, value),
            fuzz.interp_membership(universe, high, value)
            )

#Test stopnie    
def calculate_membership(data, var_name, plot=False, na_omit=True, expert=False, printout=False):
    column = data[var_name]
    result = pd.DataFrame(np.zeros(len(column)*3).reshape(-1,3))
    result.columns = [var_name + "_low", var_name + "_medium", var_name + "_high"]
    
    #for i in range(1):
    for i in range(len(column)):
        result.loc[i,] = membership_function(data, var_name, column[i], 0, 0, plot, na_omit, expert, 0)
        if printout==True:
            print(str(result.loc[i,]))
            print(str(column[i]))

            
    return result

def calculate_membership_fixed(data, var_name, plot=False, na_omit=True, expert=False, printout=False,
                              use_central_and_spread=True, central=0, spread=0.1):
    column = data[var_name]
    result = pd.DataFrame(np.zeros(len(column)*3).reshape(-1,3))
    result.columns = [var_name + "_low", var_name + "_medium", var_name + "_high"]
    
    #for i in range(1):
    for i in range(len(column)):
        result.loc[i,] = membership_function(data, var_name, column[i], central, spread, plot, na_omit, expert, use_central_and_spread) 
        if printout==True:
            print(str(result.loc[i,]))
            print(str(column[i]))

            
    return result

def quantifier_func(x):
    part = np.arange(0, 1.01, 0.001)
    majority = fuzz.trapmf(part, [0.5, 0.6, 1, 1])
    minority = fuzz.trapmf(part, [0, 0, 0.3, 0.50])
    almost_all = fuzz.trapmf(part, [0.8, 0.9, 1, 1])
    part_majority = fuzz.interp_membership(part, majority, x)
    part_minority = fuzz.interp_membership(part, minority, x)
    part_almost_all =  fuzz.interp_membership(part, almost_all, x)
    return dict(majority = part_majority, 
                minority = part_minority, 
                almost_all = part_almost_all)
   

def t_norm(a, b, ntype):
    """
    calculates t-norm for param a and b
    :param ntype:
        1 - minimum
        2 - product
        3 - Lukasiewicz t-norm
    """
    if ntype == 1:
        return np.minimum(a, b)
    elif ntype == 2:
        return a * b
    elif ntype == 3:
        return np.maximum(0, a + b - 1)

def Degree_of_truth(d, Q="majority", P="loudness_low", P2=""):
    """
    Degree of truth for short protoforms
    """
    p = np.mean(d[P])

    return quantifier_func(p)[Q]

def cxc():
    return("OLGA")

def Degree_of_truth_ext(d, Q="majority", P="loudness_medium", R="", tnorm="min"):
    """
    Degree of truth for extended protoforms
    """

    #print("dot")
    if(tnorm=="min"):
        p = np.fmin(d[P], d[R])
        #print(p)
    else:
        p = np.fmax(0,(d[P]+d[R]-1))
    
    r = d[R]
    #print(r)
    
    if np.sum(r) == 0:
        t = 0
        #print(P+"_"+R+"dot is 0")
    else:
        t = np.sum(p) / np.sum(r)
        
    return quantifier_func(t)[Q]

def Degree_of_support(d, P="loudness_medium"):
    """
    Degree of support for short protoforms informs how many objects are covered by a particular summary
    """

    DoS = sum(d[P]>0)/len(d)
    
    return DoS

def Degree_of_support_ext(d, P="loudness_medium", R="quality_low", tnorm="min"):
    """
    Degree of support for extended protoforms informs how many objects are covered by a particular summary
    """

    if(tnorm=="min"):
        p = np.fmin(d[P], d[R])
    else:
        p = np.fmax(0,(d[P]+d[R]-1))
    
    DoS = sum(p>0)/len(d)
    
    return DoS

def Degree_of_focus_ext(d, P="loudness_medium", R="quality_low"):
    """
    Degree of focus applies to extended protoforms and informs how many objects satisfy the qualifier of the particular summary
    """

    DoF = sum(d[R])/len(d)
    
    return DoF

def all_protoform(d, var_names, Q = "majority", desc = 'most', classtoprint="class"):
    """
    Function that determines the degrees of truth support and focus for all linguistic summaries (simple and complex)   
    """
    
    pp = [var_names[0] + "_low", var_names[0] + "_medium", var_names[0] + "_high"]
    qq = [var_names[1] + "_low", var_names[1] + "_medium", var_names[1] + "_high"]
    qq_shap_print = ["against predicting "+classtoprint+" class", "around zero to predicting "+classtoprint +" class", "positively to predicting "+classtoprint+" class"]
    pp_print = [var_names[0], var_names[0],var_names[0]]
    pp_print1 = ["low", "medium","high"]
    
    protoform = np.empty(9, dtype = "object")
    Id = np.zeros(9)
    DoT = np.zeros(9)
    DoS = np.zeros(9)
    DoF = np.zeros(9)
    k = 0
    
    for i in range(len(pp)):
        for j in range(len(qq)):   
            DoT[k] = Degree_of_truth_ext(d = d, Q = Q, P = pp[j], R = qq[i])
            DoS[k] = Degree_of_support_ext(d = d, P = pp[j], R = qq[i])
            DoF[k] = Degree_of_focus_ext(d = d, P = pp[j], R = qq[i])
            protoform[k] = "Among records that contribute "+ qq_shap_print[i] + ", "+ desc + " of them have " + pp_print[j] + "-related features at "+pp_print1[j]+" level."
            Id[k] = k
            k += 1
            
    dd = {'Id': Id,
          'protoform': protoform,
            'DoT': DoT,
            'DoS': DoS,
            'DoF' : DoF}
    dd = pd.DataFrame(dd)   
    return dd[['Id', 'protoform', "DoT", 'DoS', "DoF"]]

#Comparison among XGBoost, MLP baseline and Sequential compositional MLP

##XGBoost classifier

###Perfomance evaluation

In [37]:
from xgboost import XGBClassifier

In [38]:
model_name='XGBOOST'

In [39]:
#Train the XGBoost model
#We create a dictionary that contains our model hyperparameters
xgb_params = {
    'n_estimators': 200, 
    'eval_metric': 'error',
    'max_depth': 3, 
    'objective': 'binary:logistic',
}
xgb_model = XGBClassifier(**xgb_params)
xgb_model = xgb_model.fit(X_train, y_train)
y_pred=xgb_model.predict(X_test)

In [40]:
print(confusion_matrix(y_test, y_pred))

[[ 81  39]
 [ 34 167]]


In [41]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.68      0.69       120
           1       0.81      0.83      0.82       201

    accuracy                           0.77       321
   macro avg       0.76      0.75      0.76       321
weighted avg       0.77      0.77      0.77       321



In [42]:
#number of samples to consider for the explanation 
max_samples=y_test.shape[0] #10
#number of features to show in the shap graphs. Default value is 20, we can modify it.
max_features=feature_names.shape[0] #total number of features in data

###Cross validation

In [43]:
cv_scores = cross_val_score(xgb_model, X, y, cv=skf)
cv_scores.mean(), cv_scores.std()

(0.7582474226804123, 0.031158282261941315)

In [44]:
cv_scores

array([0.7755102 , 0.7244898 , 0.75257732, 0.82474227, 0.73195876,
       0.74226804, 0.74226804, 0.78350515, 0.72164948, 0.78350515])

##MLP baseline

###Performance evaluation

In [45]:
model_name='baseline'

In [46]:
# baseline neural network
def build_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(10, input_shape=(22,), activation='relu', name='dense'),
        tf.keras.layers.Dropout(0.2, name='dropout'),
        tf.keras.layers.Dense(2, activation='sigmoid', name='output')])
    
    model.build()
    
    return model

In [47]:
baseline = build_model()

In [48]:
#baseline.summary()

In [49]:
baseline.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])

In [50]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [51]:
baseline.fit(X_train, y_train, epochs=50,
             validation_data=(X_test, y_test),
             callbacks=[early_stopping])

Epoch 1/50
21/21 [==============================] - 1s 11ms/step - loss: 0.6416 - accuracy: 0.6774 - val_loss: 0.6334 - val_accuracy: 0.6511
Epoch 2/50
21/21 [==============================] - 0s 2ms/step - loss: 0.6226 - accuracy: 0.6759 - val_loss: 0.6210 - val_accuracy: 0.6573
Epoch 3/50
21/21 [==============================] - 0s 5ms/step - loss: 0.6180 - accuracy: 0.6743 - val_loss: 0.6126 - val_accuracy: 0.6386
Epoch 4/50
21/21 [==============================] - 0s 4ms/step - loss: 0.6062 - accuracy: 0.6836 - val_loss: 0.6055 - val_accuracy: 0.6449
Epoch 5/50
21/21 [==============================] - 0s 3ms/step - loss: 0.6041 - accuracy: 0.6805 - val_loss: 0.5991 - val_accuracy: 0.6542
Epoch 6/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5911 - accuracy: 0.6943 - val_loss: 0.5919 - val_accuracy: 0.6698
Epoch 7/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5934 - accuracy: 0.6836 - val_loss: 0.5850 - val_accuracy: 0.6667
Epoch 8/50
21/21 [=

In [52]:
y_pred_states = baseline.predict(X_test)
#y_pred_states

In [53]:
y_pred_states = np.argmax(y_pred_states, axis=1)
#y_pred_states

In [54]:
print(confusion_matrix(y_test, y_pred_states))

[[ 53  67]
 [ 15 186]]


In [55]:
print(classification_report(y_test, y_pred_states))

              precision    recall  f1-score   support

           0       0.78      0.44      0.56       120
           1       0.74      0.93      0.82       201

    accuracy                           0.74       321
   macro avg       0.76      0.68      0.69       321
weighted avg       0.75      0.74      0.72       321



###Explanation with Shap

We use a Kernel explainer to explain the neural network results https://shap-lrjball.readthedocs.io/en/docs_update/generated/shap.KernelExplainer.html#shap.KernelExplainer

In [56]:
red_dim=100 #size of the reduced dimensionality (big dimensions are not allowed by Shap)
#kmeans is used to reduce the dimensionality when big data are used. This is a shap requirement. It is not 
#able to process big data.
X_train_summary = shap.kmeans(X_train, red_dim) 
explainer = shap.KernelExplainer(baseline.predict, X_train_summary)

In [57]:
max_samples= X_test.shape[0]

In [58]:
shap_values = explainer.shap_values(X_test[1:max_samples,:])

  0%|          | 0/320 [00:00<?, ?it/s]

Output streaming troncato alle ultime 5000 righe.
model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: o

In [59]:
#bar plot shows the average impact on model output magnitude of all features and all classes. 
#Features are ordered in descending order by impact
shap.summary_plot(shap_values, X_test[1:max_samples,:], plot_type="bar", class_names= classes_names, feature_names = feature_names,max_display=max_features,show=False)
plt.gcf()
figname=graphsStreamPath+'_survey_'+model_name+'_global_allclasses_'+ str(max_features)+'.png'
plt.savefig(figname,dpi=300, bbox_inches='tight')
plt.clf()

<Figure size 576x741.6 with 0 Axes>

In [60]:
#one different summary plot is created for each class. It represents the impact on the model output of each feature in descending order.
for class_id in range(len(shap_values)):
    shap.summary_plot(shap_values[class_id], X_test[1:max_samples,:], feature_names = feature_names, max_display=max_features,show=False)
    plt.gcf()
    figname=graphsStreamPath+'_survey_'+model_name+'_global_class'+str(class_id)+'_features'+ str(max_features)+'.png'
    plt.savefig(figname,dpi=150, bbox_inches='tight')
    plt.clf()

<Figure size 576x741.6 with 0 Axes>

In [61]:
shap_values_0_df = pd.DataFrame(shap_values[0], columns = feature_names)
shap_values_1_df = pd.DataFrame(shap_values[1], columns = feature_names)

shap_values_0_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_0_baseline.csv", index=False)
shap_values_1_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_1_baseline.csv", index=False)


In [62]:
X_test_df.to_csv(shapvaluesStreamPath + "/data_shap_survey_baseline.csv", index=False)

###Explanation with Linguistic Summaries

In [63]:
type_of_eval=model_name
#classcode=1

In [64]:
#import data
#data = pd.read_csv(DataFile, sep=',')
data=X_test_df_nn
data = data[:-1]
#shapdata = pd.read_csv(ShapFile, sep=',')
#shapdata=shap_values_0_df

for classcode in [0,1]:
  shapdata = pd.read_csv('/content/drive/MyDrive/bipolar_shap/shapvalues/shap_values_survey_class_'+str(classcode)+'_'+type_of_eval+'.csv', sep=',')
    #shapdata.shape
  data=data.reset_index()
  shapdata=shapdata.reset_index()
  data.columns

  ####################################################################################################################################################
  # Parameters
  ####################################################################################################################################################

  plot=False
  printout=False

  if classcode==0: classtoprint='control'
  if classcode==1: classtoprint='treatment'

  expert = False #dictionary with expert opinion about
  relative_LS = False #if relative LS is True, patient_no must be provided
  spread=0.02

  ####################################################################################################################################################
  # Basic stats
  ####################################################################################################################################################

  if "var_names" in locals():
      del var_names
  #variables to be summarized
  var_names=['Age',	
  #'Gender',	
  #'self_employed',	
  #'family_history',	
  'work_interfere',	
  'no_employees',	
  #'remote_work',	
  #'tech_company',	
  'benefits',	
  'care_options',	
  'wellness_program',	
  'seek_help',	
  'anonymity',
  'leave','mental_health_consequence',	
  'phys_health_consequence',	
  'coworkers',
  'supervisor',
  'mental_health_interview',	
  'phys_health_interview',	
  'mental_vs_physical',	
  #'obs_consequence'
  ]

  predicted_var=['class']

  if plot==True:
      for zmienna in var_names:
            fig=plt.figure(figsize=(15,8))
            sns.boxplot(x="class", y=zmienna, data = data.loc[:,["class",zmienna]])
            fig.savefig(ResultsDir+"Stats_"+str(zmienna)+".png")
            fig=plt.figure(figsize=(15,8))
            #sns.boxplot(shapdata.loc[:,[zmienna]])
            #fig.savefig("Stats_shap_"+str(zmienna)+"_classtoprint_"+classtoprint+".png")

  plot=False
      
  #NA percentage
  data2=data[var_names]
  data2.columns = var_names
  data2.agg(lambda x: np.mean(x.isna())).reset_index().rename(columns={'index': 'column', 0: 'NA_percentage'})

  ######################################################################################################################
  # Linguistic summaries for individual parameters
  ######################################################################################################################

  data3_full = data2.copy()
  #data3 = data2.copy()
  data3=data2.copy()
  #data4 = shapdata.copy()
  data4 = shapdata[var_names].copy()

  if "df_protoform_all" in locals():
     del df_protoform_all
    
  #for name in var_names[0:1]:
  for name in var_names:
        print(name)
        #temp = calculate_membership(data3, name, plot,expert=expert, printout=printout)
        temp = calculate_membership_fixed(data3, name, plot,expert=False,printout=printout, use_central_and_spread=False, central=0, spread="100")
        temp2 = calculate_membership_fixed(data4, name, plot,expert=True, printout=printout, use_central_and_spread=True, central=0, spread=spread)
        temp2.columns=["shap_"+name+"_low","shap_"+name+"_medium","shap_"+name+"_high",]
        data_for_lingsum = pd.concat([temp,temp2], axis=1)
        temp_var_names=[name, "shap_"+name]
        df_protoform = all_protoform(data_for_lingsum, temp_var_names, Q = 'majority', desc = 'most', classtoprint=classtoprint)
        if "df_protoform_all" in locals():
            df_protoform_all = df_protoform_all.append(df_protoform)
        else:
            df_protoform_all = df_protoform.copy()
        data3_full = pd.concat([data3_full, data_for_lingsum], axis=1)
        data3_full.to_csv(ResultsDir+'data_with_memberships_MentalSurveys_spread_'+str(spread)+'_class_'+str(classcode)+'alg_'+type_of_eval+'.csv')
        df_protoform_all.to_csv(ResultsDir+"Protoforms_MentalSurveys33_spread_"+str(spread)+'_class_'+str(classcode)+'alg_'+type_of_eval+'.csv') 

Age
work_interfere
no_employees
benefits
care_options
wellness_program
seek_help
anonymity
leave
mental_health_consequence
phys_health_consequence
coworkers
supervisor
mental_health_interview
phys_health_interview
mental_vs_physical
Age
work_interfere
no_employees
benefits
care_options
wellness_program
seek_help
anonymity
leave
mental_health_consequence
phys_health_consequence
coworkers
supervisor
mental_health_interview
phys_health_interview
mental_vs_physical


### Cross validation

In [65]:
cv_scores = []

for train_index, test_index in skf.split(X, y):
    X_trainK, X_testK = X[train_index], X[test_index]
    y_trainK, y_testK = y[train_index], y[test_index]

    baseline = build_model()

    baseline.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
    
    baseline.fit(X_trainK, y_trainK, epochs=50,
             validation_data=(X_testK, y_testK),
             callbacks=[early_stopping])
    
    test_loss, test_acc = baseline.evaluate(X_testK, y_testK)

    cv_scores.append(test_acc)

Epoch 1/50
28/28 [==============================] - 0s 6ms/step - loss: 0.7018 - accuracy: 0.5286 - val_loss: 0.6667 - val_accuracy: 0.6531
Epoch 2/50
28/28 [==============================] - 0s 2ms/step - loss: 0.6494 - accuracy: 0.6831 - val_loss: 0.6349 - val_accuracy: 0.6633
Epoch 3/50
28/28 [==============================] - 0s 2ms/step - loss: 0.6252 - accuracy: 0.6876 - val_loss: 0.6175 - val_accuracy: 0.6837
Epoch 4/50
28/28 [==============================] - 0s 2ms/step - loss: 0.6108 - accuracy: 0.6842 - val_loss: 0.6049 - val_accuracy: 0.6735
Epoch 5/50
28/28 [==============================] - 0s 3ms/step - loss: 0.6046 - accuracy: 0.6899 - val_loss: 0.5926 - val_accuracy: 0.7041
Epoch 6/50
28/28 [==============================] - 0s 3ms/step - loss: 0.5809 - accuracy: 0.7059 - val_loss: 0.5801 - val_accuracy: 0.7143
Epoch 7/50
28/28 [==============================] - 0s 3ms/step - loss: 0.5722 - accuracy: 0.7117 - val_loss: 0.5712 - val_accuracy: 0.7245
Epoch 8/50
28/28 [==

In [66]:
mean(cv_scores), stdev(cv_scores)

(0.7376498937606811, 0.039570889867686115)

In [67]:
cv_scores

[0.7448979616165161,
 0.7346938848495483,
 0.7525773048400879,
 0.8041236996650696,
 0.7422680258750916,
 0.7628865838050842,
 0.7010309100151062,
 0.7628865838050842,
 0.7113401889801025,
 0.6597937941551208]

## Sequential compositional MLP + NMF for the middle layer

###Performance evaluation

Since the considered data do not have intermediate lables we create synthetic ones by means of Non Negative Matrix Factorization (NMF) (https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html).
It is able to extract Hidden Factors (HF) from data, and represent it as an additive linear combination of HFs.

It decomposes the original matrix in two matrices:


*   the bases matrix contains the bases of the new subspace where the data are spanned
*   the encoding matrix contains the coefficients of data in the new subspace

We will use the encoding matrix as a middle layer that represents data in terms of Hidden Factors.



In [68]:
model_name='one_head'

In [133]:
from sklearn.decomposition import NMF

#number of hidden factors we want to extract
n_clusters = 5

input = tf.keras.layers.Input(shape=(22,), name='input')
hidden = tf.keras.layers.Dense(10, activation='relu', name='dense')(input)
dropout = tf.keras.layers.Dropout(0.2, name='dropout')(hidden)
intermediate_output = tf.keras.layers.Dense(n_clusters, name='intermediate_output')(dropout)
final_output = tf.keras.layers.Dense(2, activation='sigmoid', name='final_output')(intermediate_output)

In [134]:
one_head = tf.keras.Model(inputs=input, 
                          outputs=[intermediate_output, final_output], 
                          name='one-head-model')

In [135]:
#one_head.summary()

In [136]:
one_head.compile(optimizer='adam',
                 loss=[tf.keras.losses.MeanAbsoluteError(),
                       tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)],
                 loss_weights=[0.5, 0.5],
                 metrics=['mae','accuracy'])

In [137]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_final_output_accuracy', patience=5)

In [138]:
clusterer = NMF(n_components=n_clusters, init='random', solver='cd',max_iter=1000,tol=1e-6, random_state=42)
clusterer.fit(X_train)
y_train_cluster = clusterer.transform(X_train)
y_test_cluster = clusterer.transform(X_test)
hidden_factors = clusterer.components_
#feature_groups=np.argmax(hidden_factors,axis=0)
#feature_groups_df = pd.DataFrame(feature_groups, feature_names)


In [139]:
one_head.fit(X_train, [y_train_cluster, y_train], epochs=50,
             validation_data=(X_test, [y_test_cluster, y_test]),
             callbacks=[early_stopping])

Epoch 1/50
21/21 [==============================] - 35s 13ms/step - loss: 0.6759 - intermediate_output_loss: 0.6559 - final_output_loss: 0.6958 - intermediate_output_mae: 0.6559 - intermediate_output_accuracy: 0.1966 - final_output_mae: 0.5232 - final_output_accuracy: 0.5868 - val_loss: 0.6332 - val_intermediate_output_loss: 0.5894 - val_final_output_loss: 0.6770 - val_intermediate_output_mae: 0.5894 - val_intermediate_output_accuracy: 0.1651 - val_final_output_mae: 0.5237 - val_final_output_accuracy: 0.5857
Epoch 2/50
21/21 [==============================] - 0s 4ms/step - loss: 0.6163 - intermediate_output_loss: 0.5668 - final_output_loss: 0.6657 - intermediate_output_mae: 0.5668 - intermediate_output_accuracy: 0.2012 - final_output_mae: 0.5263 - final_output_accuracy: 0.5960 - val_loss: 0.5895 - val_intermediate_output_loss: 0.5200 - val_final_output_loss: 0.6590 - val_intermediate_output_mae: 0.5200 - val_intermediate_output_accuracy: 0.1807 - val_final_output_mae: 0.5233 - val_fina

In [140]:
y_pred = one_head.predict(X_test)

In [141]:
y_pred = np.argmax(y_pred[1][:], axis=1)
y_pred

array([0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [142]:
print(confusion_matrix(y_test, y_pred))

[[ 55  65]
 [ 11 190]]


In [143]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.46      0.59       120
           1       0.75      0.95      0.83       201

    accuracy                           0.76       321
   macro avg       0.79      0.70      0.71       321
weighted avg       0.78      0.76      0.74       321



In [144]:
accuracy_score(y_test, y_pred)

0.7632398753894081

###Explanation with Shap in term of treatments (final layer)

We use predictions from the final output to explain the results through Shap.


In [145]:
def f(X):
    return one_head.predict(X)[1]# with this function we select the second output of the model: vector of states 

In [146]:
explainer = shap.KernelExplainer(f, X_train_summary) 

In [147]:
shap_values = explainer.shap_values(X_test[1:max_samples,:])

  0%|          | 0/320 [00:00<?, ?it/s]

Output streaming troncato alle ultime 5000 righe.
model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: o

In [148]:
#bar plot
shap.summary_plot(shap_values, X_test[1:max_samples,:], plot_type="bar", class_names= classes_names, feature_names = feature_names, max_display=max_features , show=False)
plt.gcf()
figname=graphsStreamPath+model_name+'_global_survey_allclasses_states_'+str(max_features)+'.png'
plt.savefig(figname,dpi=300, bbox_inches='tight')
plt.clf()

<Figure size 576x741.6 with 0 Axes>

In [149]:
#summary plots for each output class
for class_id in range(len(shap_values)):
    shap.summary_plot(shap_values[class_id], X_test[1:max_samples,:], feature_names = feature_names, max_display=max_features,show=False)
    plt.gcf()
    figname=graphsStreamPath+'_survey_'+model_name+'_global_class'+str(class_id)+'_features'+ str(max_features)+'.png'
    plt.savefig(figname,dpi=300, bbox_inches='tight')
    plt.clf()

<Figure size 576x741.6 with 0 Axes>

In [150]:
shap_values_0_df = pd.DataFrame(shap_values[0], columns = feature_names)
shap_values_1_df = pd.DataFrame(shap_values[1], columns = feature_names)

shap_values_0_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_0_onehead_states.csv", index=False)
shap_values_1_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_1_onehead_states.csv", index=False)

###Explanation with Shap in terms of symptoms (middle layer)

We use predictions from the intermediate output to explain the results through Shap.
Please note that we don't have labels, but Hidden Factors.


In [151]:
def f(X):
    return one_head.predict(X)[0]# with this function we select the first output of the model: matrix of meta-features 

In [152]:
explainer = shap.KernelExplainer(f, X_train_summary)

In [153]:
shap_values = explainer.shap_values(X_test[1:max_samples,:])

  0%|          | 0/320 [00:00<?, ?it/s]

Output streaming troncato alle ultime 5000 righe.
model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: o

In [154]:
meta_classes_names=['HF_1','HF_2','HF_3','HF_4','HF_5']

In [155]:
#bar plots
shap.summary_plot(shap_values, X_test[1:max_samples,:], plot_type="bar", class_names= meta_classes_names, max_display=max_features, feature_names = feature_names,show=False)
plt.gcf()
figname=graphsStreamPath+'_survey_'+model_name+'_global_allclasses'+str(max_features)+'.png'
plt.savefig(figname,dpi=300, bbox_inches='tight')
plt.clf()

<Figure size 576x741.6 with 0 Axes>

In [156]:
#summary plots for each intermediate class
for x in range(len(meta_classes_names)):
  meta_classes_names[x]
  shap.summary_plot(shap_values[x], X_test[1:max_samples,:], feature_names = feature_names, max_display=max_features,show=False) #you can change the maximum features to display 
  plt.gcf()
  figname=graphsStreamPath+'_survey_'+model_name+'_global_class_'+meta_classes_names[x]+'_'+str(max_features)+'.png'
  plt.savefig(figname,dpi=300, bbox_inches='tight')
  plt.clf()

<Figure size 576x741.6 with 0 Axes>

In [157]:
shap_values_0_df = pd.DataFrame(shap_values[0], columns = feature_names)
shap_values_1_df = pd.DataFrame(shap_values[1], columns = feature_names)
shap_values_2_df = pd.DataFrame(shap_values[2], columns = feature_names)
shap_values_3_df = pd.DataFrame(shap_values[3], columns = feature_names)
shap_values_4_df = pd.DataFrame(shap_values[4], columns = feature_names)


shap_values_0_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_0_onehead_HF.csv", index=False)
shap_values_1_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_1_onehead_HF.csv", index=False)
shap_values_2_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_2_onehead_HF.csv", index=False)
shap_values_3_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_3_onehead_HF.csv", index=False)
shap_values_4_df.to_csv(shapvaluesStreamPath + "/shap_values_survey_class_4_onehead_HF.csv", index=False)



In [158]:
#feature_groups_df.to_csv(shapvaluesStreamPath + "/feature_groups_survey_onehead.csv", index=True)
#hidden_factors_df= pd.DataFrame(hidden_factors, columns = feature_names)
#hidden_factors_df.to_csv(shapvaluesStreamPath + "/hidden_factors_survey_onehead.csv", index=True)

In [159]:
H_matrix_df=pd.DataFrame(y_test_cluster)
H_matrix_df.to_csv(shapvaluesStreamPath + "/H_matrix_survey_onehead.csv", index=True)

In [160]:
# we can assign each sample to the HF that mostly contributes in its definition
#y_test_metaclasses=np.argmax(y_test_cluster,axis=1)
#X_test_df['meta_class']=y_test_metaclasses
#X_test_df.to_csv(shapvaluesStreamPath + "/X_survey_onehead_metaclasses.csv", index=False)

###Explanation with Linguistic Summaries in term of treatments (final layer)

In [161]:
type_of_eval='onehead_states'

In [162]:
#import data
#data = pd.read_csv(DataFile, sep=',')
data=X_test_df_nn
data = data[:-1]
#shapdata = pd.read_csv(ShapFile, sep=',')
#shapdata=shap_values_0_df

for classcode in [0,1]:
  shapdata = pd.read_csv('/content/drive/MyDrive/bipolar_shap/shapvalues/shap_values_survey_class_'+str(classcode)+'_'+type_of_eval+'.csv', sep=',')
    #shapdata.shape
  data=data.reset_index()
  shapdata=shapdata.reset_index()
  data.columns

  ####################################################################################################################################################
  # Parameters
  ####################################################################################################################################################

  plot=False
  printout=False

  if classcode==0: classtoprint='control'
  if classcode==1: classtoprint='treatment'

  expert = False #dictionary with expert opinion about
  relative_LS = False #if relative LS is True, patient_no must be provided
  spread=0.02

  ####################################################################################################################################################
  # Basic stats
  ####################################################################################################################################################

  if "var_names" in locals():
      del var_names
  #variables to be summarized
  var_names=['Age',	
  #'Gender',	
  #'self_employed',	
  #'family_history',	
  'work_interfere',	
  'no_employees',	
  #'remote_work',	
  #'tech_company',	
  'benefits',	
  'care_options',	
  'wellness_program',	
  'seek_help',	
  'anonymity',
  'leave','mental_health_consequence',	
  'phys_health_consequence',	
  'coworkers',
  'supervisor',
  'mental_health_interview',	
  'phys_health_interview',	
  'mental_vs_physical',	
  #'obs_consequence'
  ]

  predicted_var=['class']

  if plot==True:
      for zmienna in var_names:
            fig=plt.figure(figsize=(15,8))
            sns.boxplot(x="class", y=zmienna, data = data.loc[:,["class",zmienna]])
            fig.savefig(ResultsDir+"Stats_"+str(zmienna)+".png")
            fig=plt.figure(figsize=(15,8))
            #sns.boxplot(shapdata.loc[:,[zmienna]])
            #fig.savefig("Stats_shap_"+str(zmienna)+"_classtoprint_"+classtoprint+".png")

  plot=False
      
  #NA percentage
  data2=data[var_names]
  data2.columns = var_names
  data2.agg(lambda x: np.mean(x.isna())).reset_index().rename(columns={'index': 'column', 0: 'NA_percentage'})

  ######################################################################################################################
  # Linguistic summaries for individual parameters
  ######################################################################################################################

  data3_full = data2.copy()
  #data3 = data2.copy()
  data3=data2.copy()
  #data4 = shapdata.copy()
  data4 = shapdata[var_names].copy()

  if "df_protoform_all" in locals():
     del df_protoform_all
    
  #for name in var_names[0:1]:
  for name in var_names:
        print(name)
        #temp = calculate_membership(data3, name, plot,expert=expert, printout=printout)
        temp = calculate_membership_fixed(data3, name, plot,expert=False,printout=printout, use_central_and_spread=False, central=0, spread="100")
        temp2 = calculate_membership_fixed(data4, name, plot,expert=True, printout=printout, use_central_and_spread=True, central=0, spread=spread)
        temp2.columns=["shap_"+name+"_low","shap_"+name+"_medium","shap_"+name+"_high",]
        data_for_lingsum = pd.concat([temp,temp2], axis=1)
        temp_var_names=[name, "shap_"+name]
        df_protoform = all_protoform(data_for_lingsum, temp_var_names, Q = 'majority', desc = 'most', classtoprint=classtoprint)
        if "df_protoform_all" in locals():
            df_protoform_all = df_protoform_all.append(df_protoform)
        else:
            df_protoform_all = df_protoform.copy()
        data3_full = pd.concat([data3_full, data_for_lingsum], axis=1)
        data3_full.to_csv(ResultsDir+'data_with_memberships_MentalSurveys_spread_'+str(spread)+'_class_'+str(classcode)+'alg_'+type_of_eval+'.csv')
        df_protoform_all.to_csv(ResultsDir+"Protoforms_MentalSurveys33_spread_"+str(spread)+'_class_'+str(classcode)+'alg_'+type_of_eval+'.csv') 

Age
work_interfere
no_employees
benefits
care_options
wellness_program
seek_help
anonymity
leave
mental_health_consequence
phys_health_consequence
coworkers
supervisor
mental_health_interview
phys_health_interview
mental_vs_physical
Age
work_interfere
no_employees
benefits
care_options
wellness_program
seek_help
anonymity
leave
mental_health_consequence
phys_health_consequence
coworkers
supervisor
mental_health_interview
phys_health_interview
mental_vs_physical


###Cross validation

In [163]:
cv_scores = []

for train_index, test_index in skf.split(X, y):
    X_trainK, X_testK = X[train_index], X[test_index]
    y_trainK, y_testK = y[train_index], y[test_index]
 
    
    clusterer = NMF(n_components=n_clusters, init='random', solver='cd',max_iter=1000,tol=1e-6, random_state=42)
    clusterer.fit(X_trainK)
    y_train_cluster = clusterer.transform(X_trainK)
    y_test_cluster = clusterer.transform(X_testK)


    one_head = tf.keras.Model(inputs=input, 
                              outputs=[intermediate_output, final_output], 
                              name='one-head-model')
    
    one_head.compile(optimizer='adam',
                 loss=[tf.keras.losses.MeanAbsoluteError(),
                       tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)],
                 loss_weights=[0.5, 0.5],
                 metrics=['mae','accuracy'])


    one_head.fit(X_trainK, [y_train_cluster, y_trainK], epochs=50,
                 validation_data=(X_testK, [y_test_cluster, y_testK]),
                 callbacks=[early_stopping])
    
    y_pred = one_head.predict(X_testK)
    y_pred = np.argmax(y_pred[1][:], axis=1)

    cv_scores.append(accuracy_score(y_testK, y_pred))

Epoch 1/50
28/28 [==============================] - 1s 10ms/step - loss: 0.4336 - intermediate_output_loss: 0.3320 - final_output_loss: 0.5351 - intermediate_output_mae: 0.3320 - intermediate_output_accuracy: 0.3021 - final_output_mae: 0.5300 - final_output_accuracy: 0.7311 - val_loss: 0.4362 - val_intermediate_output_loss: 0.3149 - val_final_output_loss: 0.5574 - val_intermediate_output_mae: 0.3149 - val_intermediate_output_accuracy: 0.3367 - val_final_output_mae: 0.5295 - val_final_output_accuracy: 0.7959
Epoch 2/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4225 - intermediate_output_loss: 0.3211 - final_output_loss: 0.5240 - intermediate_output_mae: 0.3211 - intermediate_output_accuracy: 0.2963 - final_output_mae: 0.5309 - final_output_accuracy: 0.7380 - val_loss: 0.4283 - val_intermediate_output_loss: 0.3012 - val_final_output_loss: 0.5553 - val_intermediate_output_mae: 0.3012 - val_intermediate_output_accuracy: 0.3469 - val_final_output_mae: 0.5301 - val_final

In [164]:
mean(cv_scores), stdev(cv_scores)

(0.7757311171891437, 0.035218142382603394)

In [165]:
cv_scores

[0.7653061224489796,
 0.7755102040816326,
 0.7628865979381443,
 0.7938144329896907,
 0.8041237113402062,
 0.7216494845360825,
 0.7525773195876289,
 0.7938144329896907,
 0.7422680412371134,
 0.845360824742268]